# Desnormalización de coordenadas

Este notebook demuestra cómo es posible desnormalizar los datos obtenidos en las predicciones realizadas por yolo.

Trsas realizar las predicciones, los resutlados quedan guardadso en runs/detect/predict/labels.

Estos txt contienen tanto la etiqueta predicha como la imagen original. Para poder seguir usando estas coordenada, será necesario desnormalizarlas.

In [1]:
import utils.yolo_fun as yolo_fun
from utils.reverse_pixel import *
import utils.img_fun as img_fun
import os
import pandas as pd
from tqdm import tqdm  
import rasterio
from rasterio.windows import Window
from rasterio.errors import RasterioIOError
import numpy as np
import shutil
from sklearn.model_selection import train_test_split


# Indicamos el input path de la imagen que queremos desnormalizar
input_img = './datasets/penguin_dataset/images/train/recorte_6_-59.21061762925279_-62.28551585752274.tiff'



# Indicamos el path con el txt de la label asociada
coords_csv = './datasets/penguin_dataset/labels/train/recorte_6_-59.21061762925279_-62.28551585752274.txt'


# Finalmente indicamos donde queremos guardar el csv con las coordenadas desnormalizadas
output_dir = 'coords/ejemplo_reverse.csv'


df_coords_normalized = pd.read_csv(coords_csv, sep=' ', header=None)


# Ejemplo de uso

A continuación vemos como la denormalización recibe los tres parámetros.

In [2]:
img_dir = './runs/detect/predict/recorte_6_-59.21074959059679_-62.28591513099474.jpg'
label_dir = './runs/detect/predict/labels/recorte_6_-59.21074959059679_-62.28591513099474.txt'
df_coords_normalized = pd.read_csv(label_dir, sep=' ', header=None)


denormalize_yolo_coords(
    tiff_file = input_img,
    coords_normalizadas = df_coords_normalized,
    output_file = output_dir
)

### Uso dentro de un bucle

Idealmente, utilizaremos la función dentro de un bucle que recorra todos los archivos de la carpeta. El nombre de cada archivo será exactamente el mismo que el de la imagen al que se refieren todas las coordenadas. 


In [3]:

#! ASEGURARSE DE QUE LAS PREDICCIONES Y LAS IMÁGENES USAN UN SISTEMA DE COORDENADSA VÁLIDO

# Aquí estarían las imágenes usadas en la predicción
img_dir = './runs/detect/predict/'
img_dir = './datasets/penguin_dataset/images/val'

# Directorio con las rutas en txt
label_dir = './datasets/penguin_dataset/labels/val'

# Output path desnormalizado
output_dir = './runs/detect/predict/labels_denormalized/'


for file in os.listdir(label_dir):
    df_coords_normalized = pd.DataFrame()
    print("Limpio", df_coords_normalized.head())
    if file.endswith('.txt') and not file.startswith('negative'):
        file_path = os.path.join(label_dir, file)
        tiff_file = os.path.join(img_dir, file.replace('.txt', '.tiff'))


        df_coords_normalized = pd.read_csv(file_path, sep=' ', header=None)
        print(f"\nProcesando archivo: {file}")
        print(f"Ruta del archivo TIFF: {tiff_file}")
        print(df_coords_normalized.head())
       
        # Llamar a la función con datos frescos
        denormalize_yolo_coords(
            tiff_file=tiff_file,
            coords_normalizadas=df_coords_normalized.copy(), 
            output_file=os.path.join(output_dir, file)
        )



Limpio Empty DataFrame
Columns: []
Index: []


EmptyDataError: No columns to parse from file

### Merge de las coordenas resultantes

Como último paso, y para seguir utilizando dichas coordenas en qgis o rasterio, bastaría con funcionarlas todas en el mismo documento.

In [12]:
output_dir = './runs/detect/predict/merged_coords.csv'
denormalized_dir = './runs/detect/predict/labels_denormalized/'

# Mergeamos todos los csv en uno solo
merged = pd.DataFrame()
for file in os.listdir(denormalized_dir):
    df = pd.read_csv(os.path.join(denormalized_dir, file), sep=' ', header=None)
    df = df.iloc[:, [1, 2]]
    merged = pd.concat([merged, df])

merged.to_csv(output_dir, index=False)
   

In [9]:
# Aquí estarían las imágenes usadas en la predicción
img_dir = './runs/detect/predict/'


# Directorio con las rutas en txt
label_dir = './runs/detect/predict/labels'

# Output path desnormalizado
output_dir = './runs/detect/predict/labels_denormalized/'

df = pd.read_csv('./runs/detect/predict/labels/recorte_6_-59.21074959059679_-62.28591513099474.txt', header=None, sep=' ')
df.head()

,0,1,2,3,4
0,0,0.908275,0.947678,0.045063,0.045025
1,0,0.781967,0.808611,0.046154,0.046348
2,0,0.909873,0.706560,0.044804,0.045108
3,0,0.793620,0.851954,0.048031,0.046318
4,0,0.878297,0.816401,0.043309,0.044051
